In [1]:
from fastcore.all import *
from fastai.vision.widgets import *
from fastai.vision.all import *

import os
from os.path import join

import PIL
import numpy as np

import matplotlib.pyplot as plt

import torch.nn
import torch
from torch.utils.data import dataloader, DataLoader

from tqdm import tqdm

In [2]:
path = untar_data(URLs.MNIST)
path

Path('C:/Users/0rc44/.fastai/data/mnist_png')

In [3]:
train_path = path/'training'
test_path = path/'test'

In [4]:
labels = list()
data = list()
for root,dirs,files in os.walk(train_path):
    for file in files:
        if file.endswith(".png"):
            img = PIL.Image.open(join(root,file))
            data.append(np.asarray(img).flatten())
            labels.append(parent_label(join(root,file)))

# data[0]

labels = np.array(labels,dtype=int)
data = np.stack(data,axis=0)

In [5]:
data.shape

(60000, 784)

In [6]:
labels.shape

(60000,)

In [7]:
train_data = data / 255
test_data = labels.reshape(-1,1)

In [8]:
X = torch.tensor(train_data)
y = torch.tensor(test_data).long()

In [9]:
dset = list(zip(X,y))

In [10]:
dl = DataLoader(dset,batch_size=5,shuffle=True)

In [11]:
X

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [12]:
y

tensor([[0],
        [0],
        [0],
        ...,
        [9],
        [9],
        [9]])

In [13]:
class Classifier(torch.nn.Module):
    def __init__(self,n_inputs,n_outputs):
        super(Classifier,self).__init__()
        self.seq = torch.nn.Sequential(
             torch.nn.Linear(in_features=n_inputs,out_features=128)
            ,torch.nn.ReLU()
            ,torch.nn.Linear(in_features=128,out_features=64)
            ,torch.nn.ReLU()
            ,torch.nn.Linear(in_features=64,out_features=n_outputs)
            ,torch.nn.Softmax(dim=1)
         
        )
        
    def forward(self,x):
        return self.seq(x.float())

In [14]:
data.shape

(60000, 784)

In [15]:
torch_model = Classifier(n_inputs = data.shape[-1],n_outputs = 10)

In [16]:
(torch_model(X).argmax(axis=1) == y.flatten()).float().mean()

tensor(0.1080)

In [17]:
epochs = 10

criterion = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(torch_model.parameters(),lr=0.01)


# training loop 


for epoch in tqdm(range(epochs)):
    total_loss = 0.0
    for _X, _label in dl: 
        y_pred = torch_model(_X)
        loss = criterion(y_pred,_label.flatten().long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:28<00:00,  8.87s/it]


In [18]:
(torch_model(X).argmax(axis=1) == y.flatten()).float().mean()

tensor(0.8637)

In [19]:
torch_model(X)

tensor([[1.0000e+00, 1.0078e-29, 2.8261e-08,  ..., 2.8414e-13, 6.0074e-15,
         6.6321e-23],
        [9.9999e-01, 1.1518e-22, 8.0416e-08,  ..., 3.3694e-11, 8.3390e-09,
         3.6563e-15],
        [9.9996e-01, 4.7007e-28, 2.3355e-09,  ..., 1.7429e-11, 3.7935e-05,
         4.3548e-15],
        ...,
        [6.1289e-16, 1.2053e-17, 2.6543e-14,  ..., 9.5521e-08, 1.5398e-08,
         1.0000e+00],
        [2.4030e-13, 5.8079e-15, 1.1008e-13,  ..., 4.0307e-03, 2.5749e-08,
         9.9597e-01],
        [1.9041e-15, 1.4735e-12, 3.0994e-12,  ..., 4.2874e-06, 5.1373e-07,
         9.9999e-01]], grad_fn=<SoftmaxBackward0>)

In [20]:
print(torch_model)

Classifier(
  (seq): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
    (5): Softmax(dim=1)
  )
)
